Tasks for collecting the Microsoft Access tables
+ Connect to the database
+ Get list of tables
+ Read each table and add to dictionary
+ For each table collected, export to a python object

In [7]:
import pandas as pd
import pyodbc # for connecting to MS Access
import pickle # for exporting
from datetime import datetime

In [13]:
# Check correct driver installed
[x for x in pyodbc.drivers() if x.startswith('Microsoft Access Driver')]

# Create connection
conn = pyodbc.connect(
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=/home/caw/g-drive/blah.accdb;')
    #r'DBQ=C:\Users\cawhi\Google Drive\Semester_3\IFN704\Project2\Data\HTS Datasets for Queensland\2017-18_QTS_ERV12_Final (2).accdb;')

# Create cursor used to execute commands
crsr = conn.cursor()

Error: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'Microsoft Access Driver (*.mdb, *.accdb)' : file not found (0) (SQLDriverConnect)")

In [ ]:
# Get a list of all the tables
for table_info in crsr.tables(tableType='TABLE'):
    print(table_info.table_name)

Now we know the tables we'd like to import, let's import them, adding to a dictionary

**Note: in previous attempts it was found that the table named 'R_TIME' returns a datetime object that pyodbc cannot handle. Consequently, that table required manual export to CSV to import a separate activity as coding the exception to handle it was technically challengings.**

In [ ]:
# Initialise dictionary to store dataframes (MS Access tables)
access_tables = {}

# Loop through the list of tables
for item in crsr.tables(tableType = 'TABLE'):
    # Remember to catch the offending table that didn't import
    if item.table_name != 'R_TIME':
        query_text = "SELECT * FROM " + item.table_name
        SQL_Query = pd.read_sql_query(query_text, conn)
        df = pd.DataFrame(SQL_Query)
        access_tables[item.table_name] = df
        
        # Just a visual to make sure we're getting the tables
        print(item.table_name)    

Next, export dataframes to local drive for importing later. Use pickel for this and the dictionary key, which we made the table name, for the filename.

In [ ]:
# This is where to store them
filepath = './Access_Tables/'
for k, v in access_tables.items():
    df = pd.DataFrame(v)
    df.to_pickle(filepath + k + ".pyobj")
    df.to_csv(filepath + k + ".csv"))

The final task is to manually import the R_TIME table that caused errors. The offending columns was 'TIME2'.

In [ ]:
column_names = ['TIME', 'STRTHR', 'STRT15', 'Peak', 'TIME2'] # Taken from MS Access

df = pd.read_csv(r"C:\Users\cawhi\Google Drive\Semester_3\IFN704\Project2\Data\HTS Datasets for Queensland\R_TIME.txt", names=column_names)
df.head()

Below are the data types. Looks like TIME2 is a string but should be a datetime. This should be converted.

In [ ]:
df.dtypes

In [ ]:
# Function that will split the string by space and convert to datetime
def correct_to_datetime(date_time):
    split_date_time = date_time.split(" ")
    tail = split_date_time[1]
    new_time = datetime.strptime(tail, '%H:%M:%S')
    return new_time

In [ ]:
# Apply to the column and take a look
df['TIME2'] = df['TIME2'].apply(correct_to_datetime)
df.head()

Everything looks good. Now to export this last table. 

In [ ]:
filename = 'R_TIME'
df.to_pickle(filepath + filename + ".pyobj")
df.to_csv(filepath + k + ".pyobj")
conn.close